In [13]:
import numpy as np
import pandas as pd
import requests
import io
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
import seaborn as sns   
from sklearn.metrics import accuracy_score
from sklearn.feature_selection import SelectKBest, SelectPercentile, f_classif


In [14]:

url = "https://cdn.jsdelivr.net/gh/ramenfeast/BV-ethnicity-report/BV%20Dataset%20copy.csv"
download = requests.get(url).content
df = pd.read_csv(io.StringIO(download.decode('utf-8')))

#%%Clean data
df = df.drop([394,395,396], axis = 0)

#%% Separate the Data and Labels
X = df.iloc[:,:-1]
y = df.iloc[:,-1]

#%% Train Test Split
X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.2, random_state=0)
#%% Extract Ethinic group and commmunity group data
es_xtest = X_test[['Ethnic Groupa']].copy()
cs_xtest = X_test[['Community groupc ']].copy()
X_test=X_test.drop(labels= ['Ethnic Groupa', 'Community groupc '], axis=1)


es_xtrain = X_train[['Ethnic Groupa']].copy()
cs_xtrain = X_train[['Community groupc ']].copy()
X_train=X_train.drop(labels= ['Ethnic Groupa', 'Community groupc '], axis=1)

#%%Normalization

#Normalize pH
X_train['pH']=X_train['pH']/14
X_test['pH']=X_test['pH']/14

#Normalize 16s RNA data
X_train.iloc[:,1::]=X_train.iloc[:,1::]/100
X_test.iloc[:,1::]=X_test.iloc[:,1::]/100

#%%Binary y
y_train[y_train<7]=0
y_train[y_train>=7]=1

y_test[y_test<7]=0
y_test[y_test>=7]=1

print(y_train)

215    0.0
145    0.0
255    0.0
331    0.0
390    0.0
      ... 
323    0.0
192    0.0
117    0.0
47     1.0
172    0.0
Name: Nugent score, Length: 315, dtype: float64


In [15]:
#Get features highest ftest
fvalue_Best = SelectKBest(f_classif, k=46)
fvalue_Best.fit(X_train, y_train)
#print(X_kbest)
#print('Original number of features:', X.shape)
#print('Reduced number of features:', X_kbest.shape)

cols = fvalue_Best.get_support(indices=True) 
features_df_newtrain = X_train.iloc[:,cols]
features_df_newtest = X_test.iloc[:,cols]
#print(features_df_new)
features_df_newtrain.info()

#print(features_df_new_target.shape)
#print(features_df_new.shape[1])

#original RF
clfrf = RandomForestClassifier(n_estimators = 100, random_state=0)
clfrf.fit(X_train, y_train)
y_pred = clfrf.predict(X_test)
print(f'Accuracy Score RF = {accuracy_score(y_test, y_pred)}')

#Ftest RF
clfrf_imp = RandomForestClassifier(n_estimators=100)
clfrf_imp.fit(features_df_newtrain,y_train)
y_pred_imp = clfrf_imp.predict(features_df_newtest)
print(f'Important Features Accuracy Score RF = {accuracy_score(y_test, y_pred_imp)}')

<class 'pandas.core.frame.DataFrame'>
Int64Index: 315 entries, 215 to 172
Data columns (total 46 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   pH                              315 non-null    float64
 1   L. iners                        315 non-null    float64
 2   L. crispatus                    315 non-null    float64
 3   L. jensenii                     315 non-null    float64
 4   Prevotella                      315 non-null    float64
 5   Megasphaera                     315 non-null    float64
 6   Sneathia                        315 non-null    float64
 7   Atopobium                       315 non-null    float64
 8   Dialister                       315 non-null    float64
 9   Lachnospiraceae_8               315 non-null    float64
 10  Anaerococcus                    315 non-null    float64
 11  Peptoniphilus                   315 non-null    float64
 12  Eggerthella                     31

/Users/dionming/opt/anaconda3/envs/BVrun/lib/python3.7/site-packages/sklearn/feature_selection/_univariate_selection.py:112: UserWarning: Features [ 95 134 208 212 231 237] are constant.
  warnings.warn("Features %s are constant." % constant_features_idx, UserWarning)
/Users/dionming/opt/anaconda3/envs/BVrun/lib/python3.7/site-packages/sklearn/feature_selection/_univariate_selection.py:113: RuntimeWarning: invalid value encountered in true_divide
  f = msb / msw


In [16]:
from scipy import stats
from statsmodels.stats.weightstats import ztest

#twotailed ttest
#loops through every column of df column into columnName and rows into X_trainfeature
#measure with mean
significantlist = []
for (columnName, X_trainfeature) in X_train.iteritems():
    #two tailed t-test
    t_value,p_value= stats.ttest_ind(X_trainfeature, y_train,equal_var=False )
    #t_value,p_value= ztest(X_trainfeature, y_train)
    print(columnName)
    #tvalue and pvalue of correlating feature and Nugent
    print('t-value:',t_value)
    print('p-value:',p_value)

    alpha = 0.01

    if p_value < alpha:
        significantlist.append(columnName)
        
print(significantlist)
print(len(significantlist))

pH
t-value: 3.654140734636928
p-value: 0.00030126810660413946
L. iners
t-value: 2.721691067561621
p-value: 0.006676177198214214
L. crispatus
t-value: -0.17857440437321387
p-value: 0.858330438975411
L. gasseri
t-value: -6.956403383594644
p-value: 1.2525945105163561e-11
L. jensenii
t-value: -7.071631193092466
p-value: 6.523901706216989e-12
Prevotella
t-value: -7.196743839613067
p-value: 3.6358319848791758e-12
Megasphaera
t-value: -8.643235382371566
p-value: 2.3591260184938445e-16
Sneathia
t-value: -8.711407645236454
p-value: 1.3538292991795894e-16
Atopobium
t-value: -8.999161693901645
p-value: 1.8831290706704685e-17
Streptococcus
t-value: -8.883623336083255
p-value: 3.5338496712131295e-17
Dialister
t-value: -9.46281057517494
p-value: 7.044135726115877e-19
Lachnospiraceae_8
t-value: -9.501180172198074
p-value: 4.738202677754006e-19
Anaerococcus
t-value: -9.602384826610841
p-value: 2.44176788611204e-19
Peptoniphilus
t-value: -9.774466176046717
p-value: 6.97571305310256e-20
Eggerthella
t-va

In [24]:
from scipy import stats

#pointbiserial
#used when one variable is interval and other variable has only 2 possible variables
significantlist = []
correlationlist = []
for (columnName, X_trainfeature) in X_train.iteritems():
    #point biserial 
    correlation_value,p_value= stats.pointbiserialr(X_trainfeature, y_train)
    print(columnName)
    #correlationvalue and pvalue of correlating feature and Nugent
    print('pearson value:',correlation_value)
    print('p-value:',p_value)

    alpha = 0.05

    if p_value < alpha:
        significantlist.append(columnName)
        #if between .5 and 1 is strongly correlated
        if correlation_value > abs(.5): 
            correlationlist.append(columnName)

print('significant list:')
print(significantlist)
print(len(significantlist))
print('correlation list:')
print(correlationlist)
print(len(correlationlist))

pH
pearson value: 0.5771818977181464
p-value: 2.2455564519252887e-29
L. iners
pearson value: -0.2713812581848008
p-value: 1.0111601111357042e-06
L. crispatus
pearson value: -0.3536924882429806
p-value: 1.0295618844112325e-10
L. gasseri
pearson value: -0.12939324846838615
p-value: 0.021616653331087064
L. jensenii
pearson value: -0.18248222680254783
p-value: 0.0011410489464043457
Prevotella
pearson value: 0.6496663163267248
p-value: 3.742571677690979e-39
Megasphaera
pearson value: 0.6414046987504098
p-value: 6.631470539399074e-38
Sneathia
pearson value: 0.5410638021528581
p-value: 2.373924160994417e-25
Atopobium
pearson value: 0.4678449640658055
p-value: 1.5465142172042286e-18
Streptococcus
pearson value: -0.029001517660701796
p-value: 0.6081000072782555
Dialister
pearson value: 0.5287278514427137
p-value: 4.383395312245677e-24
Lachnospiraceae_8
pearson value: 0.3606485218399397
p-value: 4.144367223679335e-11
Anaerococcus
pearson value: 0.1904463808977759
p-value: 0.000679299507334687
Pe